# Importing (steps 1-3)

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Importing tables
path = r'C:\Users\Lenovo\Instacart Basket Analysis'


In [3]:
cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
df_prod_ord = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_aggregated.pkl'),)

#  Wrangling (step 4)

In [5]:
# checking dataset

In [6]:
cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [7]:
cust.shape

(206209, 10)

In [8]:
# renaming columns

In [9]:
cust.rename(columns = {'Surnam': 'Last Name'}, inplace = True)

In [10]:
cust.rename(columns = {'n_dependants': 'Dependants'}, inplace = True)

In [11]:
# checking results

In [12]:
cust.head()

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,Dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [13]:
cust.shape

(206209, 10)

# Consistency Checks (step 5)

In [14]:
# checking missing values 

In [15]:
cust.isnull().sum()

user_id            0
First Name     11259
Last Name          0
Gender             0
STATE              0
Age                0
date_joined        0
Dependants         0
fam_status         0
income             0
dtype: int64

In [16]:
# dropping first name column (this information should not be relevant to our analysis)

In [17]:
df_cust = cust.drop("First Name", axis='columns')

In [18]:
df_cust.shape

(206209, 9)

In [19]:
# checking data types

In [20]:
df_cust.dtypes

user_id         int64
Last Name      object
Gender         object
STATE          object
Age             int64
date_joined    object
Dependants      int64
fam_status     object
income          int64
dtype: object

In [21]:
# chenging data types

In [22]:

df_cust['user_id'] = df_cust['user_id'].astype('str')

In [23]:
print(df_cust['user_id'].dtypes)

object


In [24]:
# checking data consistentcy (mix type)

In [26]:

for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

In [29]:
# checking duplicates


In [30]:
df_dup = df_cust[df_cust.duplicated()]

In [31]:
df_dup.shape

(0, 9)

In [32]:
# applying descriptive statistics

In [33]:
df_cust.describe()

,Age,Dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


# Combining Datasets (step 6)

In [34]:
df_prod_ord.shape

(32406041, 21)

In [35]:
# checking data consistency between both datasets

In [36]:
df_prod_ord.dtypes

order_id                     int64
user_id                      int64
eval_set                    object
order_number                 int64
order_day_of_week            int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
max_order                    int64
loyalty_flag                object
mean_price                 float64
spending_flag               object
mean_days                  float64
frequency_flag              object
dtype: object

In [37]:
# chenging data types for consistency

In [38]:
df_prod_ord['user_id'] = df_prod_ord['user_id'].astype('str')

In [39]:
print(df_prod_ord['user_id'].dtypes)

object


In [40]:
# merging data sets on 'user_id' 

In [41]:
df = df_prod_ord.merge(df_cust, on = 'user_id')

In [42]:
df.head()

,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,mean_days,frequency_flag,Last Name,Gender,STATE,Age,date_joined,Dependants,fam_status,income
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,prior,3,3,12,21.0,196,1,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,prior,5,4,15,28.0,196,1,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


# Final Consistency Check (step 7)

In [43]:
# renaming columns for consistency

In [44]:
df.rename(columns = {'Last Name' : 'last_name',  'STATE' : 'state',  'fam_status' : 'family_status'}, inplace = True)


In [45]:
df.head()

,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,mean_days,frequency_flag,last_name,Gender,state,Age,date_joined,Dependants,family_status,income
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,prior,3,3,12,21.0,196,1,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,prior,5,4,15,28.0,196,1,1,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


# Exporting Dataframe (step 8)

In [46]:
# exporting to pickle

In [48]:
df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data','cust_ord_prod.pkl'))